# Topic 19: Multiple Regression and Model Validation

## a. Making Predictions in Statsmodels

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [ ]:
df = pd.read_csv('heightWeight.csv')
plt.scatter(df.height, df.weight)
plt.xlabel("Height")
plt.ylabel("Weight")
plt.title("Scatterplot of Height vs Weight")
plt.show()

In [ ]:
f = 'weight~height'
model = ols(formula=f, data=df).fit()
model.summary()

In [ ]:
df.head()

In [ ]:
df.iloc[4]

In [ ]:
model.predict(df.iloc[4])

In [ ]:
to_predict = 4

print('Actual weight: ', df.iloc[to_predict]['weight'])
print('Predicted weight: ', model.predict(df.iloc[to_predict])[to_predict])

In [ ]:
def predict_weight(height):
    w = -204.4834 + 5.539*height
    return f"With a height of {height} inches, predicted weight of {round(w, 2)} pounds."

In [ ]:
predict_weight(60)

## b. Multiple Regression

In [ ]:
car_df = pd.read_csv('auto-mpg.csv')
car_df.rename({'model year': 'year'}, axis='columns', inplace=True)
car_df

In [ ]:
formula = 'mpg ~ cylinders+displacement+acceleration+weight+year+origin'
model = ols(formula=formula, data=car_df).fit()
model.summary()

## c. Categorical Variables

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16,3))

for xcol, ax in zip(['acceleration', 'displacement', 'horsepower', 'weight'], axes):
    car_df.plot(kind='scatter', x=xcol, y='mpg', ax=ax, alpha=0.4, color='b')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12,3))

for xcol, ax in zip([ 'cylinders', 'year', 'origin'], axes):
    car_df.plot(kind='scatter', x=xcol, y='mpg', ax=ax, alpha=0.4, color='b')

In [ ]:
cyl_dummies = pd.get_dummies(car_df['cylinders'], prefix='cyl', drop_first=True)
yr_dummies = pd.get_dummies(car_df['year'], prefix='yr', drop_first=True)
orig_dummies = pd.get_dummies(car_df['origin'], prefix='orig', drop_first=True)

In [ ]:
data = car_df.drop(['cylinders','year','origin', 'car name'], axis=1)
data = pd.concat([data, cyl_dummies, yr_dummies, orig_dummies], axis=1)
data.head()

In [ ]:
data.columns[1:]

In [ ]:
formula = 'mpg ~ '+ '+'.join(data.columns[1:])
formula

In [ ]:
model = ols(formula=formula, data=data).fit()
model.summary()

## d. Multicollinearity of Features

In [ ]:
data.corr()

In [ ]:
# first check for linearity between target and features
data.corr()[abs(data.corr()['mpg']) > 0.3]

In [ ]:
set1 = list(data.corr()[abs(data.corr()['mpg']) > 0.3].index)

In [ ]:
# running model with only features correlated with mpg
formula = 'mpg ~ '+ '+'.join(set1[1:])
model = ols(formula=formula, data=data).fit()
model.summary()

In [ ]:
data = data.loc[:,set1]
data.corr()

In [ ]:
# creating the correlation pairs
corr = data.corr().abs().stack().reset_index().sort_values(0, ascending=False)
display(corr)

# setting the index to the pair of variables
corr['pairs'] = list(zip(corr.level_0, corr.level_1))
corr.set_index(['pairs'], inplace = True)
corr.drop(columns=['level_1', 'level_0'], inplace = True)

# cc for correlation coefficient
corr.columns = ['cc']

corr.drop_duplicates(inplace=True)
corr[(corr.cc>.5) & (corr.cc<1)]

Let's drop variables that are highly correlated with other variables.

In [ ]:
data

In [ ]:
formula = 'mpg ~ horsepower+acceleration+cyl_4+yr_80+yr_82+orig_3'
formula

In [ ]:
model = ols(formula=formula, data=data).fit()
model.summary()

## e. Transforming Features

In [ ]:
car_df[['acceleration', 'displacement', 'horsepower', 'weight']].hist(figsize  = [6, 6])
plt.show()

In [ ]:
data_log = pd.DataFrame([])
data_log['logdisp'] = np.log(car_df['displacement'])
data_log['loghorse'] = np.log(car_df['horsepower'])
data_log['logweight'] = np.log(car_df['weight'])
data_log.hist(figsize  = [6, 6]);

### Log transformation

As seen in the previous lesson, a log transformation is a very useful tool when you have data that clearly does not follow a normal distribution. Log transformation can help reduce skewness when you have skewed data, and can help reducing variability of data. 

### Transformations that do not change the distribution of your data

#### Min-max scaling

When performing min-max scaling, you can transform x to get the transformed $x'$ by using the formula:

$$x' = \dfrac{x - \min(x)}{\max(x)-\min(x)}$$

This way of scaling brings all values between 0 and 1. 

#### Standardization

When 

$$x' = \dfrac{x - \bar x}{\sigma}$$

x' will have mean $\mu = 0$ and $\sigma = 1$

Note that standardization does not make data $more$ normal, it will just change the mean and the standard error!

#### Mean normalization
When performing mean normalization, you use the following formula:
$$x' = \dfrac{x - \text{mean}(x)}{\max(x)-\min(x)}$$

The distribution will have values between -1 and 1, and a mean of 0.

#### Unit vector transformation
 When performing unit vector transformations, you can create a new variable x' with a range [0,1]:
 
$$x'= \dfrac{x}{{||x||}}$$


Recall that the norm of x $||x||= \sqrt{(x_1^2+x_2^2+...+x_n^2)}$

In [ ]:
acc = car_df['acceleration']
logdisp = data_log['logdisp']
loghorse = data_log['loghorse']
logweight = data_log['logweight']

scaled_acc = (acc - min(acc)) / (max(acc) - min(acc))
scaled_disp = (logdisp - np.mean(logdisp)) / np.sqrt(np.var(logdisp))
scaled_weight = (logweight - np.mean(logweight)) / np.sqrt(np.var(logweight))
scaled_horse = (loghorse - np.mean(loghorse)) / (max(loghorse) - min(loghorse))

data_cont_scaled = pd.DataFrame([])
data_cont_scaled['acc'] = scaled_acc
data_cont_scaled['disp'] = scaled_disp
data_cont_scaled['horse'] = scaled_horse
data_cont_scaled['weight'] = scaled_weight

data_cont_scaled.hist(figsize = [6, 6]);

## f. Model Validation & Regression in Scikit-Learn

### The need for train-test split

#### Making predictions and evaluation

So far we've simply been fitting models to data, and evaluated our models calculating the errors between our $\hat y$ and our actual targets $y$, while these targets $y$ contributed in fitting the model.

The reason why we built the model in the first place, however, is because we want to predict the outcome for observations that are not necessarily in our dataset now; e.g: we want to predict miles per gallon for a new car that isn't part of our dataset, or for a new house in Boston.

In order to get a good sense of how well your model will be doing on new instances, you'll have to perform a so-called "train-test-split". What you'll be doing here, is take a sample of the data that serves as input to "train" our model - fit a linear regression and compute the parameter estimates for our variables, and calculate how well our predictive performance is doing comparing the actual targets $y$ and the fitted $\hat y$ obtained by our model.

#### Underfitting and overfitting

Another reason to use train-test-split is because of a common problem which doesn't only affect linear models, but nearly all (other) machine learning algorithms: overfitting and underfitting. An overfit model is not generalizable and will not hold to future cases. An underfit model does not make full use of the information available and produces weaker predictions than is feasible. The following image gives a nice, more general demonstration:

<img src="images/modelfit.png" width="700"> 

It is pretty straightforward that, to evaluate the model, you'll want to compare your predicted values, $\hat y$ with the actual value, $y$. The difference between the two values is referred to as the residuals. When using a train-test split, you'll compare your residuals for both test set and training set:

$r_{i,train} = y_{i,train} - \hat y_{i,train}$ 

$r_{i,test} = y_{i,test} - \hat y_{i,test}$ 

To get a summarized measure over all the instances in the test set and training set, a popular metric is the (Root) Mean Squared Error:

RMSE = $\sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_{i} - \hat y_{i})^2}$

MSE = $\frac{1}{n}\sum_{i=1}^{n}(y_{i} - \hat y_{i})^2$

Again, you can compute these for both the traing and the test set. A big difference in value between the test and training set (R)MSE is an indication of overfitting.

In [ ]:
data = pd.read_csv('auto-mpg.csv') 
data.rename({'model year': 'year'}, axis='columns', inplace=True)

acc = data['acceleration']
logdisp = np.log(data['displacement'])
loghorse = np.log(data['horsepower'])
logweight = np.log(data['weight'])

scaled_acc = (acc-min(acc))/(max(acc)-min(acc))
scaled_disp = (logdisp-np.mean(logdisp))/np.sqrt(np.var(logdisp))
scaled_horse = (loghorse-np.mean(loghorse))/(max(loghorse)-min(loghorse))
scaled_weight = (logweight-np.mean(logweight))/np.sqrt(np.var(logweight))

data_fin = pd.DataFrame([])
data_fin['acc'] = scaled_acc
data_fin['disp'] = scaled_disp
data_fin['horse'] = scaled_horse
data_fin['weight'] = scaled_weight
cyl_dummies = pd.get_dummies(data['cylinders'], prefix='cyl', drop_first=True)
yr_dummies = pd.get_dummies(data['year'], prefix='yr', drop_first=True)
orig_dummies = pd.get_dummies(data['origin'], prefix='orig', drop_first=True)
mpg = data['mpg']
data_fin = pd.concat([mpg, data_fin, cyl_dummies, yr_dummies, orig_dummies], axis=1)

### Scikit-Learn!

In [ ]:
data = pd.concat([mpg, scaled_acc, scaled_weight, orig_dummies], axis=1)
y = data[['mpg']]
X = data.drop(['mpg'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squarred Error:', train_mse)
print('Test Mean Squarred Error:', test_mse)

print('Train Root Mean Squarred Error:', train_mse**0.5)
print('Test Root Mean Squarred Error:', test_mse**0.5)